In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import re
from collections import Counter

import gensim as gsm
import pymorphy2 as pmr

from guidialogs import ffp, fdp
from textproc import rwtools
from tempscripts import codeutils as cdu
from tempscripts import corpproc as crp
from tempscripts import iopickler as iop
from tempscripts import reader_for_csv as rfc

2019-11-28 15:35:47,557 : INFO : Loading dictionaries from C:\Users\EA-ShevchenkoIS\AppData\Local\Continuum\anaconda3\lib\site-packages\pymorphy2_dicts\data
2019-11-28 15:35:48,242 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2019-11-28 15:35:48,368 : INFO : Loading dictionaries from C:\Users\EA-ShevchenkoIS\AppData\Local\Continuum\anaconda3\lib\site-packages\pymorphy2_dicts\data
2019-11-28 15:35:48,430 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [14]:
corp_it = rfc.CommonReader(fdp())
corp_it.create_readers()

# 1. Загружаем все найденные статьи:

In [11]:
ec_art_lem = iop.IOPickler(open(ffp(), mode='rb'))
print(type(ec_art_lem), len(ec_art_lem))

<class 'tempscripts.iopickler.IOPickler'> 38171


In [15]:
ec_art_lem[12345]

[['тк', '355', 'ст', '1', '1', 'п']]

In [18]:
print(corp_it[12345]['Текст документа'])

СТАВРОПОЛЬСКИЙ КРАЕВОЙ СУД
РЕШЕНИЕ
от 13 июля 2016 г. по делу N 7а-872/2016
Судья Тумасян Г.А. дело N 7а-872/2016
РЕШЕНИЕ
город Ставрополь 13 июля 2016 года
Судья Ставропольского краевого суда Мирошниченко Д.С., рассмотрев в открытом судебном заседании жалобу С.Х.Н., действующего в интересах Г.Э.О., на решение судьи Минераловодского городского суда Ставропольского края от 21 апреля 2016 года,
установил:
Постановлением заместителя начальника Северо-Кавказской оперативной таможни Ч.М.В. N <...> от 03 февраля 2016 года Г.Э.О. признан виновным в совершении административного правонарушения, предусмотренном ч. 1 ст. 16.2 КРФоАП, с назначением административного наказания в виде штрафа в размере ? стоимости товара, явившегося предметом административного правонарушения, что составило <...> руб. без конфискации предмета административного правонарушения.
Решением судьи Минераловодского городского суда Ставропольского края от 21 апреля 2016 года указанное постановление оставлено без изменения, жал

# 2. Создаем функцию для поиска всех статей КоАП особенной части (главы 5- 21) с индексацией соответствующих актов

In [52]:
def find_all_AM_docs(iop_obj):
    '''
    Function finds all administrative misdemeanor acts and collects appropriate articles from them
    '''
    holder = [] # store all found articles and documents
    errors = [] # for all errors in articles names
    index = {} # dictionary that remember correspondence between indicies in holder and indicies of acts
    for ind, doc in enumerate(iop_obj):
        int_holder = [] # store all appropriate arcticles for current document
        int_errors = [] # store errors in current document
        if ind % 5000 == 0:
            print('doc #', ind)
        for line in doc:
            code = line[0]
            if code == 'коап': # if code is appropriate
                art = line[1]
                try:
                    if (
                        re.match(r'[5-9]', art[0]) # condition for the first character in articles name
                        or
                        re.match(r'1[0-9]\.|2[01]\.', art[0:3]) # condition for the second character in articles name
                    ):
                        int_holder.append(art)
                except:
                    int_errors.append(line)
        if int_holder:
            holder.append((ind, int_holder))
            index[ind] = len(holder)-1
        if int_errors:
            errors.append((ind, int_errors))
    return errors, holder, index

# 3. Тестируем

In [53]:
AM_ers, AM_docs, index = find_all_AM_docs(ec_art_lem)
print(len(AM_ers), len(AM_docs))

doc # 0
doc # 5000
doc # 10000
doc # 15000
doc # 20000
doc # 25000
doc # 30000
doc # 35000
0 8221


In [31]:
AM_ers

[(3304, [['коап', '1', 'ст', '1', 'ч']]),
 (12317, [['коап', '2', 'ст', '1', 'ч']]),
 (24936, [['коап', '1', 'ст', '2', 'ч']])]

In [50]:
ec_art_lem[12317]

[['коап', '1.5', 'ст', '4', 'ч'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '17.7', 'ст'],
 ['коап', '2', 'ст', '1', 'ч'],
 ['коап', '24.1', 'ст'],
 ['коап', '26.1', 'ст'],
 ['коап', '30.17', 'ст', '2', 'ч'],
 ['тк', '192', 'ст']]

In [54]:
AM_docs[index[12317]]

(12317,
 ['17.7',
  '17.7',
  '17.7',
  '17.7',
  '17.7',
  '17.7',
  '17.7',
  '17.7',
  '17.7',
  '17.7'])

In [55]:
print(corp_it[12317]['Текст документа'])

СТАВРОПОЛЬСКИЙ КРАЕВОЙ СУД
РЕШЕНИЕ
от 5 апреля 2017 г. по делу N 7а-357/2017
Судья Ширяев А.С.
Дело N 7а - 357/2017
РЕШЕНИЕ
г. Ставрополь 05 апреля 2017 года
Судья Ставропольского краевого суда Корнушенко О.Р., рассмотрев в открытом судебном заседании жалобу представителя СМУП "ФАУН" Ф. на постановление судьи Октябрьского районного суда города Ставрополя Ставропольского края от 15 февраля 2017 года,
установил:
постановлением судьи Октябрьского районного суда города Ставрополя Ставропольского края от 15 февраля 2017 года, юридическое лицо - Ставропольское муниципальное унитарное предприятие "ФАУН" (далее - СМУП "ФАУН") признано виновным в совершении административного правонарушения, предусмотренного статьей 17.7 Кодекса Российской Федерации об административных правонарушениях, с назначением наказания в виде административного штрафа в размере 50000 (пятьдесят тысяч) рублей.
В жалобе в краевой суд представитель СМУП "ФАУН" Ф. просит отменить постановление суда и прекратить производство по

## Есть предположение, что многие акты по дарминистративным делам, связаны с трудовыми правоотношениями лишь косвенно (нормы ТК РФ упоминаются в них в связи с событием административного правонарушения, но не являются ключевыми ни для состава АП, ни для разрешения спора). Поисследуем корпус, а именно частотность терминов.

# 4. Загрузим обработанный корпус с inverted index

In [56]:
ec_all = crp.load_previous_session(fdp())

Dicts loaded:   0 min,  0.047 sec
Loading bglm
Iterator bglm loaded:   0 min, 21.813 sec
Loading bgtk
Iterator bgtk loaded:   0 min, 22.192 sec
Loading ii
Iterator ii loaded:   0 min,  1.667 sec
Loading lm
Iterator lm loaded:   0 min,  9.434 sec
Loading lm_set
Iterator lm_set loaded:   0 min,  6.375 sec
Loading tk
Iterator tk loaded:   0 min, 17.805 sec
Loading tk_set
Iterator tk_set loaded:   0 min,  9.386 sec
InIndex loaded:   0 min,  0.132 sec
Loading time:   1 min, 29.150 sec


In [73]:
ec_all.keys()

dict_keys(['bglm', 'bgtk', 'ii', 'lm', 'lm_set', 'tk', 'tk_set', 'invind', 'dcts', 'stats'])

In [74]:
ii = ec_all['invind']

In [77]:
fq = {key:len(ii[key]) for key in ii}

In [85]:
len(fq)

186394

In [80]:
cnt = Counter()
cnt.update(fq)

In [86]:
cnt.most_common(300)

[('от', 38171),
 ('в', 38171),
 ('по', 38171),
 ('и', 38170),
 ('с', 38170),
 ('на', 38169),
 ('суд', 38166),
 ('не', 38160),
 ('о', 38151),
 ('суда', 38148),
 ('к', 38121),
 ('дела', 38045),
 ('что', 38044),
 ('при', 37981),
 ('а', 37835),
 ('г', 37703),
 ('решение', 37675),
 ('для', 37639),
 ('жалобы', 37508),
 ('об', 37457),
 ('из', 37264),
 ('делу', 37251),
 ('соответствии', 37231),
 ('руководствуясь', 37209),
 ('за', 37156),
 ('без', 37110),
 ('доводы', 36651),
 ('жалобе', 36299),
 ('судьи', 35932),
 ('судебном', 35707),
 ('заседании', 35528),
 ('или', 35466),
 ('изменения', 35366),
 ('согласно', 35226),
 ('судья', 35109),
 ('как', 35093),
 ('также', 35061),
 ('том', 34821),
 ('требований', 34771),
 ('его', 34713),
 ('дело', 34613),
 ('года', 34575),
 ('связи', 34547),
 ('основании', 34265),
 ('жалобу', 34226),
 ('ст', 33893),
 ('права', 33729),
 ('удовлетворения', 33605),
 ('материалы', 33602),
 ('кодекса', 33471),
 ('п', 33447),
 ('решения', 33357),
 ('поскольку', 33328),
 ('рос